# Using this notebook to Evaluate Gemma vs Gemma Model

In [ ]:
with open(r'/kaggle/input/gemma-vs-gemma-dataset/responses_gemma_first_batch_diff_prompt_2.txt') as f:
    only_gemma = f.readlines()
    m = ''.join(only_gemma)

# print(len(only_gemma))

segments = m.split("For the question")

# Process each segment
final_list = []
for i, segment in enumerate(segments):
    if segment:  
        final_list.append(segment.strip())

In [4]:
with open(r'/kaggle/input/gemma-vs-gemma-dataset/responses_gemma_second_batch_diff_prompt_2.txt') as f:
    only_gemma = f.readlines()
    m = ''.join(only_gemma)

# print(len(only_gemma))

segments = m.split("For the question")

for i, segment in enumerate(segments):
    if segment:  
        final_list.append(segment.strip())


In [5]:
end_list = []
for i in range(0,len(final_list),3):
    end_list.append(final_list[i]+final_list[i+1]+final_list[i+2])

In [6]:
len(end_list)

48

In [7]:
print(end_list[0])

"Total of 0.06 and -1977321735.", Gemma 1's answer is"Total of 0.06 and -1977321735.", Gemma 1's answer is Total of 0.06 and -1977321735.

The total is not 0.06, and the result should be 0.06. and Gemma 2's answer is Total of 0.06 and -1977321735.

The total is not 0.06, and the result should be 0.06..These are the answers from the two agents. Now re-evaluate and analyse. Give the final answer.

The total should be 0.06, and the answer should be Total of 0.06 and -1977321735. and Gemma 2's answer is"Total of 0.06 and -1977321735.", Gemma 1's answer is Total of 0.06 and -1977321735.

The total is not 0.06, and the result should be 0.06. and Gemma 2's answer is Total of 0.06 and -1977321735.

The total is not 0.06, and the result should be 0.06..These are the answers from the two agents. Now re-evaluate and analyse. Give the final answer.

The total should be 0.06, and the answer should be Total of 0.06 and -1977321735..


## Judge for Gemma Vs Gemma Summary results

In [4]:
!pip install accelerate

In [5]:
from transformers import LlamaForCausalLM, LlamaTokenizer,AutoTokenizer, AutoModelForCausalLM
import torch
from transformers import BitsAndBytesConfig

In [6]:
class AgentManager:
    def __init__(self):
        self.agents = {}

    def create_agent(self, model_name, nick_name):
        config = BitsAndBytesConfig(load_in_8bit=True)
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name,config = config, device_map = "auto")
        self.agents[nick_name] = {'model': model, 'tokenizer': tokenizer}
        print(f"Agent '{nick_name}' created successfully.")

    def interact_with_agent(self, agent_name, prompt):
        # if agent_name not in self.agents:
        #     print(f"Agent '{agent_name}' not found.")
        #     return None
        print("Agent present")
        agent = self.agents[agent_name]
        input_ids = agent['tokenizer'](prompt, return_tensors="pt").to('cuda' if torch.cuda.is_available() else 'cpu')
        outputs = agent['model'].generate(**input_ids, max_length=2048)
        # decode(outputs[0])
        return agent['tokenizer'].decode(outputs[0])

In [7]:
from huggingface_hub import login
login()

In [9]:
agent = AgentManager()
agent.create_agent("google/gemma-2b-it",'gemma_judge')

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Agent 'gemma_judge' created successfully.


In [29]:
not_found = 0
interanswer = []
final_answers = []
for response in end_list:
  # current_input = "You are a judge here. Based on all these interactions and final answers, you are asked to judge the correct answer. The conversation is :" + response +"Think and give your final answer as a judge."
#   current_input = f"You have been provided with responses from two agents regarding a mathematical calculation:{response}.Questions for your evaluation: Based on the data provided, which agent do you think has given the correct answer? Now analyse and give the answer"

  text = agent.interact_with_agent('gemma_judge', response)
  interanswer.append(text)
  parts = text.split("Give the final answer.")

  # Check if there is any text after the phrase
  if len(parts) > 1:
      # Print the text after the phrase
      final_answers.append(parts[-1].strip())  # Using strip() to remove any leading/trailing whitespace
  else:
      print("The phrase 'Give the final answer.' is not found in the text.")
      not_found += 1


Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present


In [32]:
print(len(final_answers))

48


In [36]:
interanswer[0]

'<bos>"Total of 0.06 and -1977321735.", Gemma 1\'s answer is"Total of 0.06 and -1977321735.", Gemma 1\'s answer is Total of 0.06 and -1977321735.\n\nThe total is not 0.06, and the result should be 0.06. and Gemma 2\'s answer is Total of 0.06 and -1977321735.\n\nThe total is not 0.06, and the result should be 0.06..These are the answers from the two agents. Now re-evaluate and analyse. Give the final answer.\n\nThe total should be 0.06, and the answer should be Total of 0.06 and -1977321735. and Gemma 2\'s answer is"Total of 0.06 and -1977321735.", Gemma 1\'s answer is Total of 0.06 and -1977321735.\n\nThe total is not 0.06, and the result should be 0.06. and Gemma 2\'s answer is Total of 0.06 and -1977321735.\n\nThe total is not 0.06, and the result should be 0.06..These are the answers from the two agents. Now re-evaluate and analyse. Give the final answer.\n\nThe total should be 0.06, and the answer should be Total of 0.06 and -1977321735..<eos>'

In [33]:
final_answers

['The total should be 0.06, and the answer should be Total of 0.06 and -1977321735..<eos>',
 'Sure, here is the final answer:\n\nThe final answer is 436273717..<eos>',
 'The final answer is 4061960..<eos>',
 "Sure, here's the final answer:\n\nThe difference between -57 and -18192.801 is a very large number because the numbers are very different in magnitude..<eos>",
 'The final answer is 309256902..<eos>',
 "Sure, here's the final answer:\n\n-1348127738 + 0.02 = -1348127736..<eos>",
 "Sure, here's the final answer:\n\nThe final answer is 96498216..<eos>",
 "Sure, here's the final answer:\n\n-86.3 + (-24310967) = -24311054..<eos>",
 "Sure, here's the final answer:\n\n-44.472 + -2881 = -2836.528..<eos>",
 "Sure, here's the final answer:\n\n-50582729 and -67 are the same number..These are the answers from the two agents.<eos>",
 "Sure, here's the final answer:\n\n-119 is 119 less than -0.00675958..<eos>",
 "Sure, here's the final answer:\n\n47.3039 - 0.4 = 47.2999..<eos>",
 "Sure, here's 

In [35]:
import os
directory = os.path.dirname('/kaggle/working/responses_gemma_judge_m1.txt')
if not os.path.exists(directory):
    os.makedirs(directory)

with open('/kaggle/working/responses_gemma_judge_m1.txt', "w") as file:
    # Write each element of the list to the file
    for item in final_answers:
        file.write(item + "\n")

print("List saved to", directory)

List saved to /kaggle/working


## Evaluation Check

In [14]:
data = ['-1977321734.94',
 '436273717',
 '4062036.93',
 '18135.801',
 '30927.427689',
 '-1348127737.98',
 '-96508225',
 '-24311053.3',
 '-2925.472',
 '50582662',
 '118.99324042',
 '46.9039',
 '-95280.8',
 '-93684378',
 '837.996917',
 '1704809972.7',
 '9807982',
 '63014.937',
 '-0.315003381',
 '3258771.6',
 '-158369554',
 '19836.527',
 '-35.215',
 '-9176.392',
 '-5.2105',
 '481.915',
 '687.2854',
 '515184894',
 '610126466',
 '38748.1',
 '-672.666',
 '4643002113.89',
 '44.839133',
 '-11.867941945',
 '231370',
 '-26.2112085',
 '245120.81',
 '-25475686',
 '-0.124293',
 '675.3344',
 '-1690639841.2',
 '-847.8729613',
 '3668014',
 '-5.3850076',
 '-887088.4012',
 '-8020.359',
 '-1220.12',
 '16165811508.8',
 '53026.6012',
 '16.154078',
 '2468.2',
 '-233285.8',
 '0.1596703',
 '41306.78',
 '189067.75',
 '128798808',
 '-86.72152',
 '-73.5303105',
 '-38573.68386',
 '0.1231664',
 '-435.48717805',
 '-3616.3858',
 '-0.3970914',
 '-2342.79',
 '140731.4',
 '22678879.74',
 '-0.64320768',
 '-38.862456',
 '-504164953.3',
 '-7060104',
 '39255284',
 '-992.9031',
 '-2188654532',
 '-37496723.2',
 '-242.23231',
 '7288.07',
 '-30389.806',
 '-2681.778961',
 '-330018792.4',
 '-707150.33',
 '-20.9665',
 '-21467.07806',
 '154792.643',
 '-4126.2942',
 '1249.11553',
 '3655.842',
 '-306.94566',
 '-3066.51',
 '-45284572.8',
 '-0.9819013',
 '-2043.4621',
 '2842.77772',
 '-5.10757509',
 '9102.798',
 '-4.52298',
 '-344910',
 '-5.31228173793',
 '826.303169',
 '601040.14',
 '55873341438.3',
 '271.9284',
 '-2128734365.4',
 '141551.838',
 '1430529061',
 '-0.080382',
 '42.4624',
 '-145539.044',
 '889447.3372',
 '49.6177008',
 '80483',
 '80960.2855',
 '75.595136',
 '677790',
 '-2.493687',
 '1701.04815182',
 '-74.32832',
 '69.811419',
 '24534533.0313',
 '616213.13',
 '265158.969922',
 '-9.887',
 '90155',
 '329.611',
 '180629.2',
 '575442552',
 '1712636',
 '9468.898',
 '-4950589',
 '-883597.624',
 '25.89776',
 '215069.7669',
 '25897001.66',
 '-1447887',
 '247290.4941',
 '82769114.22',
 '0.50667808',
 '-322675643',
 '-48.385',
 '-83.39427671',
 '-146114.176',
 '911145.44',
 '14.696353',
 '41118.28',
 '-45404123',
 '12671491290',
 '306.5',
 '249909',
 '17752.99',
 '-581631',
 '0.9472168456',
 '6002235360',
 '136386978.6',
 '21744329.21',
 '58881308.6',
 '-123.73577',
 '2294.4501',
 '128.2866',
 '-3.53893863',
 '58529.49',
 '731.01736',
 '-60149313.7',
 '-657305',
 '0.66984',
 '-2503062028',
 '-269080',
 '2217311',
 '656.04217',
 '4.014683704',
 '3685624',
 '175698',
 '-155926257.8',
 '1786575.36',
 '-6431205',
 '5028.5041',
 '9.88002727',
 '67371753',
 '-30061540.7175',
 '-180680083',
 '1614211',
 '-1295647508',
 '205527977.95',
 '72015145',
 '-43.29827',
 '3.3586118',
 '-86559850.8',
 '129630.15',
 '-2891079.08066',
 '-7086328.972',
 '7249477',
 '-1507.9662',
 '3.1515364',
 '-621680.76',
 '680367',
 '3.80973',
 '7981.0075',
 '-883.2',
 '17.96017',
 '-382560746.4',
 '-327.182',
 '38740',
 '-12402.1946',
 '18096',
 '1750907825.7',
 '-171752022',
 '-78089.7',
 '-26728',
 '0.063725',
 '-4093349',
 '245559892459',
 '48582643219.6',
 '-2742389',
 '201639',
 '42746814',
 '-9563.01588',
 '-40257',
 '15357316.83',
 '2434937.3',
 '-3.57234257',
 '-2328.5882',
 '9460.068',
 '12443.4',
 '-148852450.5',
 '-295.580926',
 '-27652.319',
 '-27111283.5',
 '86684801.6',
 '2145.34',
 '21399358.52',
 '518536911',
 '-325686',
 '1169820.9',
 '2373615803.6',
 '6822',
 '-19268.1368',
 '888.052747',
 '-9.989455',
 '35993.7',
 '-2138189',
 '5418318',
 '56656093',
 '-1273.5033',
 '679270.6391',
 '1975753067.8',
 '-85395.725',
 '1690.117',
 '29075556.9',
 '-12921583',
 '-16880080.1',
 '8698231.828',
 '0.735503',
 '-124582',
 '36374.9',
 '-3089980.95',
 '4356127.81',
 '117367',
 '0.089067556129',
 '332404',
 '5125050',
 '6534.7868',
 '-806317.78',
 '3013317.8',
 '-441002.94',
 '-19937.41',
 '-38.9344765',
 '-634.963',
 '6721.0491',
 '5564.5',
 '15592715.8',
 '0.0143893',
 '16991.68057',
 '29108',
 '-19003495708',
 '137445.736',
 '43.846',
 '6858.2739',
 '25277.29867',
 '9961.2428',
 '-13864456.8',
 '24142.76041',
 '-195.09',
 '12.777898805',
 '-100.499728',
 '33851748',
 '-1100937.89',
 '-20.8061702',
 '877278.204',
 '-9640',
 '-62908',
 '-2.6879078',
 '550288.9658',
 '14674300.15',
 '-13198835.2',
 '335686451.8',
 '4089822.2',
 '57535302.13',
 '6445920',
 '-0.869992692',
 '645597140',
 '-0.009618061',
 '-41.0284671',
 '-565199.43',
 '-290831345',
 '-144536',
 '-62366.931',
 '-4.544307072',
 '-153.939',
 '-30890',
 '1809.95913',
 '305484.6',
 '-1216419176',
 '-0.50211266403',
 '8652.27214',
 '-137757.657',
 '-532556.857438',
 '-4626.7675',
 '9.976254',
 '14467.40691',
 '-21580299',
 '-17792.84455',
 '578.779147',
 '-9725.42',
 '47372543',
 '-431685',
 '140186618.62',
 '-4618006.76',
 '-15245976',
 '-4935.888',
 '32386.84',
 '38.1769',
 '-51843.91',
 '34166.5',
 '-11609785',
 '196.46017',
 '-710134.892',
 '6997',
 '-44963.7',
 '-29.202007',
 '4079515',
 '42531998252.3',
 '-309059.87',
 '-10651109',
 '-125656.6',
 '489761813.3',
 '350230.5',
 '-976080',
 '2435575',
 '-86.0389247',
 '798.65281831',
 '1.473284',
 '-4263031508.5',
 '-0.06196',
 '27823.2088',
 '732462',
 '-5113379460.1',
 '81392.346',
 '829480460',
 '5546.3315',
 '-11131947008.8',
 '770.0021',
 '0.00617392',
 '-27789626',
 '611800',
 '-1612959.77',
 '7025525.9',
 '-133347.28',
 '128291008',
 '410538.62',
 '-14613814',
 '7577479',
 '494.212',
 '8791',
 '-0.144817',
 '18.3678995',
 '-30730986.16',
 '-348.4904',
 '1491747.9',
 '20154',
 '-0.22551357',
 '7203183.91',
 '-961.53065',
 '72171198.4',
 '-12835311',
 '297810996.979',
 '39115',
 '210298.56',
 '-12358988735.1',
 '32789326.706',
 '-1508.31',
 '197863348.066',
 '-1874360',
 '0.0082',
 '158006115.6',
 '7221233112.1',
 '-417496.2',
 '-1041386311',
 '-239465.324',
 '48554895',
 '119623.8',
 '-32.680511',
 '-210.207',
 '61324.88',
 '-184261',
 '5.5366142',
 '32087323.9',
 '13.1093502',
 '296.43104',
 '-8596163.8385',
 '-135885362.5',
 '-357148599.1',
 '-12511211.962',
 '1.7577558',
 '-20208337.59',
 '666962',
 '415933',
 '-6356.336',
 '-196.954614',
 '4685.01996',
 '14142.85',
 '-1191500',
 '-305268052.29',
 '-4953617.7',
 '37655174',
 '-295.53888',
 '293497503.5',
 '-2406.1688',
 '-922.761',
 '-26376912',
 '-1501.6',
 '194.3506041',
 '6369776698.7',
 '39421.04896',
 '158695',
 '-1063132.40435',
 '272887',
 '2283859',
 '433650.686',
 '-5.352256',
 '-159.624112',
 '2.186247',
 '95959',
 '301776461',
 '-517876674.5',
 '-335296',
 '3091061.057',
 '68528.896',
 '-5916673.8',
 '-1725983748.7',
 '170900316',
 '21074.6902',
 '485769168.81',
 '-21398.9',
 '-1369587.5',
 '-1775886',
 '5828.327637',
 '-942.55479',
 '-37002',
 '31236802410',
 '224.799',
 '-23830',
 '-49.545622',
 '67431.8286',
 '-8011466641.9',
 '-829424',
 '-2456674',
 '-4525217.7',
 '-1.5746713',
 '-2445156444',
 '-3081.55',
 '-2281418.2',
 '-228025185',
 '607.621',
 '13136034.4',
 '1.17418',
 '4091526281',
 '5.952186',
 '-798.8239',
 '2044',
 '-6587911.34',
 '-28025.261',
 '-613.2845',
 '-64745509.1',
 '46375.3235',
 '175259',
 '-5300179',
 '13.307629',
 '7681.8338',
 '3250.73459',
 '15.26342',
 '1503',
 '266.563',
 '115034',
 '-920645.283',
 '-2281752',
 '-12548013',
 '240.172203',
 '-12.97006737323',
 '-1211.3721',
 '-636.8727928',
 '-12665.01704',
 '37244.653',
 '-24067789472',
 '-7724077.93695',
 '5035614741.4',
 '14721.07',
 '37110',
 '-432978',
 '3368.1959',
 '0.237139',
 '-20.44419',
 '-37357.117',
 '128.3068',
 '4028.76931',
 '408007.1',
 '-276024496.95',
 '-25429208',
 '-861.77',
 '406303',
 '-344.9243',
 '64742.6472',
 '26709456.1',
 '-123420.49379',
 '24346.8',
 '118779',
 '-7668400.4',
 '59.3287481',
 '0.22746711',
 '-15468.9',
 '1.0458789084',
 '33965524',
 '6.9145752',
 '32.80282',
 '7804679',
 '-162.183',
 '7918354316.6',
 '-224970',
 '119363.9761',
 '-2.58387',
 '29485077234',
 '59390.3727',
 '-334946',
 '0.63553486',
 '6.117883557',
 '240310',
 '-4203414',
 '14889.6',
 '-924.23456',
 '-67274.02677',
 '-201.366',
 '-0.9764896487',
 '-779.6653131',
 '57326428.75',
 '1340255.76',
 '102477',
 '-1320.8',
 '4017.038',
 '-307.299',
 '651736.97169',
 '-487805.7',
 '33.5050997',
 '-4141149006',
 '0.00316',
 '-0.3941535027',
 '-1.00893',
 '-2117.75655',
 '-25810619',
 '691.751402',
 '81646',
 '-412653070',
 '-176287',
 '-2115287563.6',
 '8329824.4',
 '-25082092',
 '-57524.9607',
 '-16779602.5',
 '58993.5919',
 '-72.49546',
 '0.7878605',
 '7145.210183',
 '8661253',
 '24729234',
 '878727',
 '3173563',
 '1732.123',
 '-4729621.9352',
 '-32312.84',
 '20480128.39',
 '230.583548',
 '119387.26',
 '135.885523',
 '31831.6',
 '-1278594282.3',
 '1.17385819',
 '-88091148',
 '-18930.9',
 '-913601127.7',
 '-12946611161.2',
 '-3.8677',
 '-83908.437',
 '-7876541.59',
 '311.16513',
 '-4.365843045',
 '-25810817.3',
 '-37.93702267',
 '15875705.3',
 '339537.018',
 '9002053',
 '-1507.3105',
 '-3986.7',
 '-16563.63',
 '736681',
 '887937410.82',
 '-6099.25',
 '323724.088',
 '1452367.12',
 '34829369',
 '-459696',
 '-1666354332.1',
 '151392.35',
 '136268509',
 '-3418883.076',
 '13.0250914',
 '15487.48032',
 '-616.71073',
 '-10891952',
 '-4103961',
 '-122348.096',
 '12663959',
 '-161.8495',
 '-16957435.08',
 '87.7022',
 '-32403.73513',
 '-734.596',
 '18.483819',
 '-1473775302',
 '14154.61798',
 '769.28973',
 '-4348.2',
 '-61576161',
 '177149.007',
 '1758944',
 '37.29444066',
 '386174.6',
 '268934',
 '-23.6879',
 '1313.265',
 '-22225738',
 '-278875784',
 '-1166421841',
 '2639.7878',
 '-91063676.2',
 '6450.08421',
 '-22552.67747',
 '5613552',
 '-0.3411734',
 '0.9622286274',
 '-9789105.6',
 '0.4238486',
 '269.961',
 '-0.21232779',
 '-1722362',
 '-17.41405739',
 '4863560',
 '328475',
 '-640697',
 '42.6232144',
 '-9.79866519',
 '136910815.6',
 '495661429.2',
 '-1912.031',
 '569470766.17',
 '-12077.236625',
 '-102374.2',
 '4.86312796',
 '10365',
 '-258659.728',
 '5067719.59',
 '-4532598',
 '-81341811.3',
 '6.41202176',
 '-113851',
 '-65699.8',
 '-138.04606',
 '3196.5',
 '-2952747.0595',
 '-11718.462',
 '-5364503.25',
 '2470050.0089',
 '-254.83256',
 '432728363.98',
 '24.714876',
 '-136269',
 '-0.54935561',
 '43.2103',
 '-111.06094',
 '871244.9542',
 '97515',
 '-1948.121',
 '152.134238',
 '-217557192',
 '1537.705',
 '754.2',
 '122458',
 '-4398',
 '7807.152',
 '5636764',
 '-492.69925043',
 '-539.2796583',
 '-638796657',
 '-2096198',
 '-834616.2',
 '-301924',
 '-5020603.06',
 '-45673.926642',
 '20943.295',
 '-5533.461',
 '18674848',
 '13537.692',
 '1814617974',
 '-65.78252236',
 '144328.2',
 '-857606027.4',
 '-140602',
 '-229610.7',
 '-174.5724603',
 '-19994801973.4',
 '93698906',
 '-641444431.022',
 '-172117073.95',
 '-406003.677',
 '25304840',
 '-195225.853',
 '121556',
 '-34784.33',
 '-35.1562493',
 '139.130378',
 '-1.8610944217',
 '918275.4581',
 '155683.6993',
 '-49839.13',
 '-10.393769267',
 '-95587',
 '442.2099918',
 '1737.07',
 '391138',
 '-158461.2',
 '8.9790075',
 '184708.07',
 '7492926',
 '830393093',
 '12361.2',
 '6329',
 '12430197.1',
 '0.42760413472',
 '625.4444004',
 '-231913386',
 '1041.91260188',
 '70937.2',
 '5.040876088',
 '-185998.089',
 '272980.073',
 '-738095.31',
 '463.529',
 '-11.7954',
 '-2339330.888',
 '-292.8791',
 '110296507.2',
 '-522164',
 '-18156.433',
 '-92360',
 '-82524859.8',
 '17612320',
 '9089.121',
 '94089543.2',
 '-2328517.16',
 '4940.794',
 '8718.15',
 '585485.2001',
 '2178.662',
 '2537.606',
 '105486681.43',
 '-1.743512153',
 '14806469.9',
 '-206.565',
 '-75954401',
 '-884.3901',
 '25501.542',
 '0.7933804',
 '-17432389.3',
 '-5727',
 '1335760.2078',
 '-4643194.2965',
 '169687448.2',
 '-34209610.37',
 '-1764.753',
 '899659.2',
 '158056325',
 '84214.5',
 '-695760',
 '-446992.381',
 '-370536',
 '-300247.45',
 '597497706',
 '-958942495',
 '643.67386',
 '-685294712.5',
 '-794389.27',
 '11133',
 '60158',
 '-21418653.99',
 '-1766472209',
 '1632871.315',
 '-31966.9713',
 '827982.29',
 '2.81037212',
 '-16.62353818',
 '-0.542113832',
 '4457.72',
 '370.3976115',
 '84619.36',
 '-66.0382',
 '-79321402',
 '36322',
 '-32984066.5',
 '-23957.8078',
 '-14938338.1',
 '73490605',
 '3942',
 '19012505.7',
 '-3706.99',
 '0.67178356',
 '1.6392541298',
 '1174450',
 '111411.45',
 '3038412155.84',
 '-1878515.46',
 '414774.033',
 '-389.6679',
 '-23.436288',
 '1328.02',
 '-5400367010',
 '-170.901',
 '7679136',
 '18646325496',
 '3875.316',
 '1.54875',
 '-1636677.25',
 '94.4665433',
 '-4657411.4',
 '-119.35589395',
 '18.6285',
 '4.485110516',
 '1508009',
 '0.428398626',
 '-27.01456',
 '-5.019114388',
 '-116116.094',
 '-174310957',
 '0.2537520865',
 '86559.5',
 '-49659.99733',
 '-1103.577335',
 '-6.2665979938',
 '-5048.75515',
 '15298.502',
 '15.330218',
 '397985',
 '-32930538.8',
 '423839835.8',
 '86421',
 '-4361621',
 '-16.078',
 '1504895.89',
 '1648.244',
 '-13476.4',
 '-2151528',
 '406575035.8',
 '1599356.2',
 '20482081.2',
 '-2179282.179',
 '-216661055',
 '-261041632.5',
 '-188650',
 '15405.98',
 '-2.89379527',
 '62196164',
 '-32068.72',
 '2927995',
 '-25888.4354',
 '76002353.04',
 '37.836713',
 '2572351522',
 '1021611.085',
 '-31954.088',
 '43030.057',
 '10176',
 '46591',
 '-842073279',
 '-90.06196',
 '-800273',
 '-79682293',
 '1221516.97',
 '-25.913',
 '-80174',
 '-2816.376833',
 '-106.394709',
 '-108.00746',
 '6584089',
 '-91455.83',
 '1093463.98',
 '345814.1',
 '-2128.586',
 '-214132',
 '-1577',
 '5.7439',
 '62354.1546',
 '-56.80428',
 '489.76668',
 '425355073.7',
 '0.3264825',
 '1.68434595',
 '177248069.698',
 '174625575.018',
 '-1.97804',
 '-66146.3',
 '630150832.8',
 '-517206.717',
 '-44834.0591',
 '1.33101419',
 '-386086042.67',
 '-3358960',
 '3185699.7',
 '-0.16886',
 '1036',
 '-3.5091625',
 '5.05269454',
 '26364.47',
 '-6183282191',
 '-1111.2905',
 '-0.69479935',
 '-470970',
 '-50745741',
 '-178.7679272',
 '-185534',
 '-10521107',
 '-0.479806',
 '-1286346',
 '6000415.2',
 '-1763727.3104',
 '-197.84762',
 '-783.1803',
 '4422720146.05',
 '-269685992.9',
 '82.039498',
 '-383979',
 '-31.971678',
 '60253742',
 '-40.4917',
 '-99.137839',
 '-49806.508',
 '389245',
 '2528187',
 '-96699.9129',
 '-1868966',
 '-680046.7',
 '-1839133',
 '-20835.477',
 '-2099.387406',
 '1446591',
 '0.005005139',
 '1093817836',
 '72.512782',
 '-117609325',
 '-129.64883587',
 '131006.957',
 '5141550.3',
 '2064248.07171',
 '0.8155',
 '-83.11893628',
 '-24654.31014',
 '-95.58443',
 '-12.361882',
 '-0.7398845',
 '865.94131',
 '-24524424.05',
 '-9994.517',
 '-889774.3',
 '25459818.79',
 '1407747.3',
 '-147.57936',
 '-1509.4718',
 '-5514.79568',
 '2.168943',
 '-6455288',
 '-3.9800104769',
 '9682606.0466',
 '22.73581',
 '2429.3205',
 '-73050.77',
 '64.14666',
 '-209440579']



In [ ]:
original = data[:48]

In [ ]:
for i in range(0,len(final_answers)):
    if original[i] in final_answers[i]:
        print(i)
    

## Judge for Summary and cross Feed method

In [2]:
with open(r'/kaggle/input/gemma-vs-gemma-dataset/responses_gemma_first_batch_diff_prompt.txt') as f:
    only_gemma = f.readlines()
    m = ''.join(only_gemma)

# print(len(only_gemma))

segments = m.split("For the question")
final_list = []
for i, segment in enumerate(segments):
    if segment:  
        final_list.append(segment.strip())
end_list = []
for i in range(0,len(final_list),3):
    end_list.append(final_list[i]+final_list[i+1]+final_list[i+2])

In [13]:
not_found = 0
interanswer = []
final_answers = []
for response in end_list:
  # current_input = "You are a judge here. Based on all these interactions and final answers, you are asked to judge the correct answer. The conversation is :" + response +"Think and give your final answer as a judge."
#   current_input = f"You have been provided with responses from two agents regarding a mathematical calculation:{response}.Questions for your evaluation: Based on the data provided, which agent do you think has given the correct answer? Now analyse and give the answer"

  text = agent.interact_with_agent('gemma_judge', response)
  interanswer.append(text)
  parts = text.split("Give the final answer.")

  if len(parts) > 1:
      final_answers.append(parts[-1].strip())  
  else:
      print("The phrase 'Give the final answer.' is not found in the text.")
      not_found += 1

Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present
Agent present


In [16]:
count = 0
original = data[:len(final_answers)]
for i in range(0,len(final_answers)):
    if original[i] in final_answers[i]:
        print(i)
        count += 1

1
20


In [17]:
print(f"The accuracy is {(count/len(final_answers))*100}")

The accuracy is 8.0
